In [1]:
import numpy as np
import copy

In [2]:
with open("input_day_18.txt") as file:
    
    num_strs = [x.strip() for x in file.readlines()]
    
    
print(num_strs)



['[[[4,5],[[0,6],[4,5]]],[3,[[5,0],[0,8]]]]', '[[8,3],2]', '[[4,[7,[5,6]]],[[[7,8],5],[[7,0],1]]]', '[[[1,8],[7,6]],[[8,6],[3,2]]]', '[[[4,[2,0]],[1,[7,0]]],9]', '[2,[[[2,3],5],[6,5]]]', '[9,[1,[0,3]]]', '[5,[5,[8,[8,4]]]]', '[5,[1,[4,[0,8]]]]', '[1,[[[6,1],9],2]]', '[7,[[6,1],[[7,8],[4,2]]]]', '[[[[6,6],[3,3]],[6,[7,6]]],4]', '[[[3,[9,8]],[[6,6],[9,3]]],[[[9,2],3],[[7,6],0]]]', '[[[[5,2],6],[9,[1,7]]],[[9,9],[9,[4,3]]]]', '[[[7,6],[9,5]],[[[6,3],[8,4]],[[4,0],8]]]', '[[[0,[1,9]],[8,[4,4]]],1]', '[[1,[1,[9,4]]],[5,[[9,3],9]]]', '[[[1,3],[[2,3],9]],[7,9]]', '[[8,[[6,9],[5,9]]],[5,[5,[9,4]]]]', '[[[[3,7],[8,0]],[4,[8,9]]],[[[3,8],[3,5]],[9,0]]]', '[[[0,5],[5,1]],[3,[0,[0,5]]]]', '[7,[[4,[1,6]],0]]', '[[3,[4,4]],[[[0,5],9],[8,[9,5]]]]', '[[8,[5,2]],[[[7,4],[3,2]],4]]', '[[[[6,4],[7,9]],5],[3,[[4,3],[4,3]]]]', '[[[[7,0],6],[6,7]],[[[9,7],[3,7]],[[4,1],[0,6]]]]', '[[6,[[1,0],[1,7]]],[3,[3,0]]]', '[[[2,[6,0]],4],[[3,9],[4,1]]]', '[[[0,[8,4]],[[8,7],5]],[1,6]]', '[[[[4,0],7],9],[6,[8,[9,3]]]]

In [3]:
class SnailfishNum:
        
    def __init__(self, snail_num_str=None, parent=None):
        if snail_num_str == None:
            pass
        elif snail_num_str[0] != "[": # ordinary number
            self.ordinary_num = int(snail_num_str[0]) # should always be one digit at initialization
            self.left = None
            self.right = None
            self.parent = parent

        else:
            depth = 0
            for i in range(len(snail_num_str)):
                if snail_num_str[i] == "[":
                    depth += 1
                elif snail_num_str[i] == "]":
                    depth -= 1
                    
                if depth == 1 and snail_num_str[i] == ",":
                    dividing_index = i
                    break
            
            self.left = SnailfishNum(snail_num_str[1 : dividing_index], self)
            self.right = SnailfishNum(snail_num_str[dividing_index + 1 : -1], self)
            self.parent = parent
            self.ordinary_num = None
            
    def __repr__(self):
        if self.parent == None:
            #print("(no parent), ", end="")
            pass
        if self.ordinary_num != None:
            return str(self.ordinary_num)
        else:
            return "[" + self.left.__repr__() + "," + self.right.__repr__() + "]"
            
    def max_depth(self):
        if self.ordinary_num != None:
            return 0
        else:
            return 1 + max(self.left.max_depth(), self.right.max_depth())
        
    def max_num(self):
        if self.ordinary_num != None:
            return self.ordinary_num
        else:
            return max(self.left.max_num(), self.right.max_num())
            
    def __add__(self, other):
        new = SnailfishNum()
        new.left = self
        new.right = other
        new.left.parent = new
        new.right.parent = new
        new.parent = None
        new.ordinary_num = None
        return new.reduced()
    
    def leftmost(self): # returns the leftmost ordinary num Snailfish num within
        if self.ordinary_num != None:
            return self
        else:
            return self.left.leftmost()
    
    def rightmost(self): # returns the rightmost ordinary num Snailfish num within
        if self.ordinary_num != None:
            return self
        else:
            return self.right.rightmost()
    
    def left_num(self): # returns the ordinary num Snailfish num directly to the left
        #print("getting left num", self)
        if self.parent == None:
            return None
        if self == self.parent.left:
            return self.parent.left_num()
        else:
            return self.parent.left.rightmost()
        
    def right_num(self): # returns the ordinary num Snailfish num directly to the right
        #print("getting right num", self)
        if self.parent == None:
            return None
        if self == self.parent.right:
            return self.parent.right_num()
        else:
            return self.parent.right.leftmost()
        
    def explode(self, max_allowable_depth):
        
        #print("exploding", self)
        
        if self.ordinary_num != None: # this must be on the left and not be pair, move on to the right looking for a pair
            self.parent.right.explode(max_allowable_depth)
        elif self.max_depth() > max_allowable_depth and self.left.ordinary_num != None and self.right.ordinary_num != None: # this is the pair that must be exploded
            #print("exploding inner")
            #print("before parent", self.parent)
            if self.left_num() != None:
                self.left_num().ordinary_num += self.left.ordinary_num
            if self.right_num() != None:
                self.right_num().ordinary_num += self.right.ordinary_num
            new = SnailfishNum("0", self.parent)
            if self.parent != None:
                if self == self.parent.left:
                    self.parent.left = new
                elif self == self.parent.right:
                    self.parent.right = new
                else:
                    print(" no parent found for explosion")
                    
            #print("after parent", new.parent)
        elif self.left.max_depth() > max_allowable_depth - 1:
            #print("exploding left")
            self.left.explode(max_allowable_depth - 1)
            return
        elif self.right.max_depth() > max_allowable_depth - 1:
            #print("exploding right")
            self.right.explode(max_allowable_depth - 1)
        else:
            print("exploding failed")
            print(self)
            print(self.max_depth(), self.left.max_depth(), self.right.max_depth(), max_allowable_depth)
            print("end failed exploding")
            
        #print("done exploding", self)
    
    def split(self, max_allowable_num):
        #print("splitting", self)

        if self.ordinary_num != None and self.ordinary_num > max_allowable_num:
            #print("splitting ", self.ordinary_num, "into ", self.ordinary_num // 2, "and ", self.ordinary_num - self.ordinary_num // 2)
            
            new = SnailfishNum()
            new.parent = self.parent
            new.ordinary_num = None
            
            newleft = SnailfishNum()
            newleft.ordinary_num = self.ordinary_num // 2
            newleft.parent = new
            newleft.left = None
            newleft.right = None
            
            newright = SnailfishNum()
            newright.ordinary_num = self.ordinary_num - self.ordinary_num // 2
            newright.parent = new
            newright.left = None
            newright.right = None
            
            new.left = newleft
            new.right = newright
            
            if self == self.parent.left:
                self.parent.left = new
            else:
                self.parent.right = new
        elif self.left.max_num() > max_allowable_num:
            self.left.split(max_allowable_num)
        elif self.right.max_num() > max_allowable_num:
            self.right.split(max_allowable_num)
            
    def reduced(self):
        
        max_allowable_depth = 4
        max_allowable_num = 9
        
        max_steps = 10000
        i = 0
        
        while self.max_depth() > max_allowable_depth or self.max_num() > max_allowable_num:
            if i > max_steps:
                print("aborted")
                break
            i += 1
            
            #print(self)
        
            if self.max_depth() > max_allowable_depth: # must be exploded
                self.explode(max_allowable_depth)
            elif self.max_num() > max_allowable_num: # must be split
                self.split(max_allowable_num)
                
        return self
    
    def magnitude(self):
        if self.ordinary_num != None:
            return self.ordinary_num
        else:
            return 3 * self.left.magnitude() + 2 * self.right.magnitude()
            
        
            

In [4]:
n = SnailfishNum("[[1,2],3]")
print(n)
print(n.left)
print(n.right.parent)

[[1,2],3]
[1,2]
[[1,2],3]


In [5]:
print(n.max_depth())
print(n.left.max_depth())
print(n.right.max_depth())

2
1
0


In [6]:
x = SnailfishNum("[[[[4,3],4],4],[7,[[8,4],9]]]")
y = SnailfishNum("[1,1]")

z = x + y

print(z)


[[[[0,7],4],[[7,8],[6,0]]],[8,1]]


In [7]:
final = SnailfishNum(num_strs[0])
for s in num_strs[1:]:
    print(final)
    final += SnailfishNum(s)
print(final)
print(final.magnitude())

[[[4,5],[[0,6],[4,5]]],[3,[[5,0],[0,8]]]]
[[[[9,0],[5,5]],[[6,7],[0,0]]],[[[8,8],3],2]]
[[[[5,6],[6,0]],[[7,7],[6,7]]],[[[6,0],[6,6]],[[7,6],[6,8]]]]
[[[[6,6],[7,7]],[[7,7],[8,7]]],[[[7,0],[8,8]],[[8,7],[7,7]]]]
[[[[7,7],[0,7]],[[7,7],[8,8]]],[[[7,7],[7,7]],[7,8]]]
[[[[7,7],[6,0]],[[7,7],[7,7]]],[[[5,6],[6,6]],[[0,8],[6,5]]]]
[[[[7,6],[0,7]],[[7,7],[7,8]]],[[[6,6],[6,6]],[1,[0,3]]]]
[[[[6,8],[6,8]],[[9,0],[9,9]]],[8,[[6,7],[0,8]]]]
[[[[8,6],[7,9]],[[9,0],[9,9]]],[[6,7],[1,[4,0]]]]
[[[[8,7],[8,9]],[[8,0],[8,7]]],[7,[[5,0],7]]]
[[[[8,6],[6,8]],[[6,6],[0,7]]],[[6,[6,7]],[[0,7],[7,6]]]]
[[[[7,7],[7,7]],[[7,7],[7,7]]],[[[0,8],[8,9]],[[5,5],[5,6]]]]
[[[[7,7],[7,7]],[[7,7],[7,7]]],[[[8,8],[8,7]],[[9,5],[8,0]]]]
[[[[6,7],[7,7]],[[7,7],[7,0]]],[[[7,7],[7,7]],[[8,8],[8,7]]]]
[[[[6,7],[7,7]],[[7,7],[7,7]]],[[[7,7],[8,0]],[[8,8],[8,7]]]]
[[[[7,7],[7,7]],[[7,7],[0,7]]],[[[7,7],[7,7]],[[5,6],[5,6]]]]
[[[[6,7],[7,9]],[[7,9],[5,6]]],[[[9,9],[5,5]],[[9,7],[0,8]]]]
[[[[6,6],[6,6]],[[6,7],[7,0]]],[[[7,8]

In [8]:
largest = 0
for x in num_strs:
    for y in num_strs:
        largest = max(largest, (SnailfishNum(x) + SnailfishNum(y)).magnitude())

print(largest)

4659
